# SMA analysis
Purpose: find best representation of SMA such that there is little correlation with actual price

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../data/BTCUSDT-day-features.csv")

In [3]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,...,macd,rsi,bb_bbm,bb_bbh,bb_bbl,bb_bbhi,bb_bbli,bb_bbw,bb_bbp,atr
0,2018-03-05,11515.00,11710.0,11415.01,11454.00,15144.231063,1520294399999,1.748670e+08,164318,7668.823714,...,191.911804,59.007457,10533.1750,11810.936738,9255.413262,0.0,0.0,24.261664,0.860327,865.953730
1,2018-03-06,11455.00,11455.0,10555.48,10716.48,29515.572363,1520380799999,3.238422e+08,250306,13482.297196,...,128.321879,51.938843,10596.4995,11774.929509,9418.069491,0.0,0.0,22.241874,0.550907,868.351321
2,2018-03-07,10716.48,10899.0,9389.31,9910.00,50647.671080,1520467199999,5.148080e+08,396721,24022.451231,...,28.740152,45.517703,10591.9950,11780.158111,9403.831889,0.0,0.0,22.435115,0.213004,914.161227
3,2018-03-08,9910.00,10099.0,9060.00,9271.64,41109.473226,1520553599999,3.946413e+08,336514,20257.455838,...,-77.516779,41.178164,10547.5780,11857.226873,9237.929127,0.0,0.0,24.833168,0.012870,923.078282
4,2018-03-09,9267.07,9410.0,8329.00,9227.00,64112.291407,1520639999999,5.670829e+08,462712,31166.414848,...,-144.320936,40.884625,10456.9505,11865.035994,9048.865006,0.0,0.0,26.931092,0.063254,934.358405
